#Necessary Imports

In [1]:
from ast import Pass
import math
from itertools import combinations
from math import isclose
import pandas as pd

In [2]:
!pip install nltk


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import wordnet, pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /home/tanmay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tanmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from ir_explain.pairwise.axioms import pairwise

[nltk_data] Downloading package stopwords to /home/tanmay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tanmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from ir_explain.pairwise.explain_more import explain_more

#Showcase some features of the ir_explain.pairwise framework:

*   TREC-DL-2019







In [7]:
!pip install ir_datasets


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


#Get some query from the list to test and display the text of the query

In [8]:
import ir_datasets
dataset = ir_datasets.load("msmarco-passage/trec-dl-2019")
query_id = "183378"

query_text = None
for query in dataset.queries_iter():
    if query.query_id == query_id:
        query_text = query.text
        break
if query_text is not None:
    print(f"Query ID: {query_id}\nQuery Text: {query_text}")
else:
    print(f"No query found with ID: {query_id}")


Query ID: 183378
Query Text: exons definition biology


#Get the top 25 ranked documents for the given query

In [9]:
target_query_id = "183378"

target_query_docs = []

for scoreddoc in dataset.scoreddocs_iter():
    if scoreddoc.query_id == target_query_id:
        target_query_docs.append((scoreddoc.doc_id, scoreddoc.score))
i=1
for doc_id, score in target_query_docs:
    print(f"Rank: {i}, Doc ID: {doc_id}")
    i+=1
    if i==26:
      break


Rank: 1, Doc ID: 7135097
Rank: 2, Doc ID: 8022481
Rank: 3, Doc ID: 7173250
Rank: 4, Doc ID: 8440424
Rank: 5, Doc ID: 719550
Rank: 6, Doc ID: 8458194
Rank: 7, Doc ID: 5210173
Rank: 8, Doc ID: 7221921
Rank: 9, Doc ID: 8073356
Rank: 10, Doc ID: 8079968
Rank: 11, Doc ID: 7248228
Rank: 12, Doc ID: 8507869
Rank: 13, Doc ID: 5259281
Rank: 14, Doc ID: 5260519
Rank: 15, Doc ID: 7275691
Rank: 16, Doc ID: 7278405
Rank: 17, Doc ID: 8529539
Rank: 18, Doc ID: 5286892
Rank: 19, Doc ID: 8139400
Rank: 20, Doc ID: 7315393
Rank: 21, Doc ID: 8562900
Rank: 22, Doc ID: 8563145
Rank: 23, Doc ID: 8570255
Rank: 24, Doc ID: 8577216
Rank: 25, Doc ID: 5331361


#Lets take the top ranked document and test it against 5th ranked document

In [10]:
target_doc_id = "7135097"

doc1 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc1 = doc.text
        break

if doc1 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc1}")
else:
    print(f"No document found with ID {target_doc_id}")


Body of Document ID 7135097:
Herbivory: Definition & Examples 4:44 Next Lesson. Allometric Growth: Definition & Example; Isometric Growth: Definition & Example; Go to AP Biology - Animal Behavior: Help and Review


In [11]:
target_doc_id = "719550"

doc2 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc2= doc.text
        break

if doc2 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc2}")
else:
    print(f"No document found with ID {target_doc_id}")


Body of Document ID 719550:
Chitin (biology) definition,meaning online encyclopedia mimi. (2) a polysaccharide which makes up the cell walls of fungi and exoskeleton of insects a tough, semitransparent substance that is the main component of the exoskeletons of art


# Applying Preprocessing on the docs

In [12]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_string(input_string):

    words = word_tokenize(input_string)

    processed_words = [ps.stem(word) for word in words if word.lower() not in stop_words]

    return ' '.join(processed_words)


# Preprocess the strings
doc1 = preprocess_string(doc1)
doc2 = preprocess_string(doc2)
query_text = preprocess_string(query_text)



#Applying Selected Axioms to see which doc is preffered


*   TFC1: assigns higher scores
to documents that contain a query term more often normalised by document length

*   LB1:If there is a query term t with tf(t, d1) = 0
and tf(t, d2) > 0, then d1 < d2.

*   LNC1:  two documents are compared that have the same term frequency
for all query terms (10%-softening). Then the shorter document is preferred.

*   AND: from a document pair d1 and d2 where only d1 contains all
query terms, d1 is preferred.

*   PROX1-3: The closer the query terms appear to each other in a document, the more relevant the document is.

*   DIV: If the Jaccard coefficient J(d,q) for document d1 is lower than J(d,q) for document d2, then d1 is preferred over d2.










In [14]:
index_path = ""

# Create an instance of the pairwise class
pairwise_instance = pairwise(query_text, doc1, doc2, index_path)

axiom_classes = [pairwise_instance.TFC1(), pairwise_instance.LB1(), pairwise_instance.AND(), pairwise_instance.DIV(), pairwise_instance.PROX1()]

pairwise_instance.explain(query_text, doc1, doc2, axiom_classes)

,Query,Document 1,Document 2,TFC1,LB1,AND,DIV,PROX1
0,exon definit biolog,herbivori : definit & exa...,chitin ( biolog ) definit...,1,0,0,-1,0


In [17]:
pairwise_instance = pairwise(query_text, doc1, doc2, index_path)

axiom_classes = [4*pairwise_instance.LNC1() + 3*pairwise_instance.TFC1(), (-1)*pairwise_instance.DIV(), pairwise_instance.PROX1() + pairwise_instance.PROX2() + pairwise_instance.PROX3() ]

pairwise_instance.explain(query_text, doc1, doc2, axiom_classes)

,Query,Document 1,Document 2,CombinedAxiom,ScaledAxiom
0,exon definit biolog,herbivori : definit & exa...,chitin ( biolog ) definit...,0,1


More details on the following axiom -


*   TFC1
*   PROX1
*   DIV





In [18]:
axiom_name = "TFC1"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Term Frequency of query terms in document1 is 4
Term Frequency of query terms in document2 is 2
0


Thus its clear from the tf values that doc1 is preferred over doc2

In [19]:
axiom_name = "PROX1"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

                      Metric  Document 1  Document 2
0                   tf(exon)    0.000000         0.0
1                tf(definit)    3.000000         1.0
2                 tf(biolog)    1.000000         1.0
3    avg_dist(exon, definit)    8.500000         0.0
4     avg_dist(exon, biolog)    0.000000         0.0
5  avg_dist(definit, biolog)    8.500000         0.0
6                  num pairs    3.000000         3.0
7             Total_avg_dist    5.666667         0.0


document 2 doesnt contain all the query terms thus as per the axiom rules the axiom doesnt prefer any document

In [20]:
axiom_name = "DIV"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Jaccard Co-efficient of doc1 is:0.09090909090909091
Jaccard Co-efficient of doc2 is:0.07692307692307693
0


Since the Jaccard Co-efficient of doc2 is lesser its preffered as seen